In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

c:\Users\tomsl\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\tomsl\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\tomsl\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Specify time window
days_in_past = 1
time_window = f'{days_in_past}' + 'D'

In [3]:
# Specify the data file here
df = pd.read_csv("./dataset_mood_smartphone.csv")

FileNotFoundError: [Errno 2] No such file or directory: './dataset_mood_smartphone.csv'

In [ ]:
# Removing the Unnamed column from the dataframe
df.drop(columns="Unnamed: 0", inplace=True)

In [ ]:
# Converting features to date and time
df["time"] = pd.to_datetime(df['time'])
df["date"] = pd.to_datetime(df['time'].dt.date)

In [ ]:
# Processing Quantiles
variables_list = df['variable'].unique()

for var in variables_list:
    # Select the variable
    screen_values = df[df['variable'] == var]['value']

    # Calculate Q1, Q3, and IQR
    Q1 = screen_values.quantile(0.05)
    Q3 = screen_values.quantile(0.95)
    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 2 * IQR

    # Filter out outliers
    filtered_screen_values = screen_values[(screen_values >= lower_bound) & (screen_values <= upper_bound)]

    # Directly filtering df
    df = df[(df['variable'] != var) | ((df['variable'] == var) & (df['value'] >= lower_bound) & (df['value'] <= upper_bound))]


In [ ]:
# Pivot the DataFrame based on id and date and average values on a day level
df_pivoted = df.pivot_table(index=['id', 'date'], columns='variable', values='value', aggfunc=['mean', 'count', 'sum']).reset_index()

# Renaming columns to remove multi-level index
df_pivoted.columns = [f"{var}{agg}" for var, agg in df_pivoted.columns]
df_pivoted

,id,date,meanactivity,meanappCat.builtin,meanappCat.communication,meanappCat.entertainment,meanappCat.finance,meanappCat.game,meanappCat.office,meanappCat.other,...,sumappCat.travel,sumappCat.unknown,sumappCat.utilities,sumappCat.weather,sumcall,sumcircumplex.arousal,sumcircumplex.valence,summood,sumscreen,sumsms
0,AS14.01,2014-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
1,AS14.01,2014-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,AS14.01,2014-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,2.0
3,AS14.01,2014-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0
4,AS14.01,2014-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,AS14.33,2014-05-27,0.012704,5.004143,59.382943,62.347000,NaN,NaN,NaN,17.835750,...,NaN,NaN,56.173,NaN,1.0,-3.0,2.0,31.0,4089.879001,2.0
1969,AS14.33,2014-05-28,0.103301,7.718042,37.238750,43.907571,NaN,NaN,21.254,27.891895,...,NaN,NaN,30.666,NaN,10.0,0.0,6.0,41.0,7551.971998,1.0
1970,AS14.33,2014-05-29,0.169354,6.452161,54.388125,68.369000,NaN,NaN,NaN,9.734000,...,0.939,NaN,3.199,NaN,5.0,-4.0,3.0,21.0,560.093000,1.0
1971,AS14.33,2014-05-30,0.192901,11.251162,46.412278,37.174286,NaN,NaN,NaN,10.522000,...,1052.648,8.072,232.825,NaN,4.0,-4.0,-2.0,34.0,8649.867999,NaN


In [ ]:
# This are the features now on a daily level
features = df_pivoted.columns.to_list()
features.remove('id')
features.remove('date')
print(len(features))

57


In [ ]:
### Creating Target  and target date columns ###

# Sort data by ID and date to ensure proper alignment
df_pivoted.sort_values(by=['id', 'date'], inplace=True)

# Create a new column by shifting the 'feature' column by -1 within each 'id' group
df_pivoted['Target'] = df_pivoted.groupby('id')['meanmood'].shift(-1)

# Similarly, shift the 'date' column to see the date of the target feature
df_pivoted['Target_date'] = df_pivoted.groupby('id')['date'].shift(-1)

In [ ]:
# Displaying dataframe
df_pivoted

,id,date,meanactivity,meanappCat.builtin,meanappCat.communication,meanappCat.entertainment,meanappCat.finance,meanappCat.game,meanappCat.office,meanappCat.other,...,sumappCat.utilities,sumappCat.weather,sumcall,sumcircumplex.arousal,sumcircumplex.valence,summood,sumscreen,sumsms,Target,Target_date
0,AS14.01,2014-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-18
1,AS14.01,2014-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-19
2,AS14.01,2014-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,2.0,NaN,2014-02-20
3,AS14.01,2014-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,2014-02-21
4,AS14.01,2014-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2014-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,AS14.33,2014-05-27,0.012704,5.004143,59.382943,62.347000,NaN,NaN,NaN,17.835750,...,56.173,NaN,1.0,-3.0,2.0,31.0,4089.879001,2.0,8.2,2014-05-28
1969,AS14.33,2014-05-28,0.103301,7.718042,37.238750,43.907571,NaN,NaN,21.254,27.891895,...,30.666,NaN,10.0,0.0,6.0,41.0,7551.971998,1.0,7.0,2014-05-29
1970,AS14.33,2014-05-29,0.169354,6.452161,54.388125,68.369000,NaN,NaN,NaN,9.734000,...,3.199,NaN,5.0,-4.0,3.0,21.0,560.093000,1.0,6.8,2014-05-30
1971,AS14.33,2014-05-30,0.192901,11.251162,46.412278,37.174286,NaN,NaN,NaN,10.522000,...,232.825,NaN,4.0,-4.0,-2.0,34.0,8649.867999,NaN,7.0,2014-05-31


In [ ]:
# Drop if there is no next day
df_pivoted.dropna(subset='Target_date', inplace=True)

In [ ]:
# Ensure target_date is in datetime format
df_pivoted['date'] = pd.to_datetime(df_pivoted['date'])

# Setting the index to target_date for rolling computation
df_pivoted.set_index('date', inplace=True)

# Applying rolling mean to all features
for feature in features:
    df_pivoted[f'time_window_{feature}'] = df_pivoted.groupby('id')[feature].transform(lambda x: x.rolling(time_window, closed='right').mean())

df_pivoted.reset_index(inplace=True)

In [ ]:
# Displaying df_pivoted
df_pivoted

,date,id,meanactivity,meanappCat.builtin,meanappCat.communication,meanappCat.entertainment,meanappCat.finance,meanappCat.game,meanappCat.office,meanappCat.other,...,sumappCat.utilities,sumappCat.weather,sumcall,sumcircumplex.arousal,sumcircumplex.valence,summood,sumscreen,sumsms,Target,Target_date
0,2014-02-17,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-18
1,2014-02-18,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-19
2,2014-02-19,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,2.0,NaN,2014-02-20
3,2014-02-20,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,2014-02-21
4,2014-02-21,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2014-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,2014-05-26,AS14.33,0.115201,7.711316,34.320472,68.876900,NaN,NaN,130.530,16.845500,...,NaN,NaN,10.0,-2.0,0.0,27.0,7193.945999,3.0,6.2,2014-05-27
1942,2014-05-27,AS14.33,0.012704,5.004143,59.382943,62.347000,NaN,NaN,NaN,17.835750,...,56.173,NaN,1.0,-3.0,2.0,31.0,4089.879001,2.0,8.2,2014-05-28
1943,2014-05-28,AS14.33,0.103301,7.718042,37.238750,43.907571,NaN,NaN,21.254,27.891895,...,30.666,NaN,10.0,0.0,6.0,41.0,7551.971998,1.0,7.0,2014-05-29
1944,2014-05-29,AS14.33,0.169354,6.452161,54.388125,68.369000,NaN,NaN,NaN,9.734000,...,3.199,NaN,5.0,-4.0,3.0,21.0,560.093000,1.0,6.8,2014-05-30


In [ ]:
# Normalize and exclude by column name and data type
def normalize_numeric(x):
    if x.name not in ['Target', 'date'] and x.dtype in ['int64', 'float64']:
        return (x - x.min()) / (x.max() - x.min()) * 2 - 1
    else:
        return x

# Apply the function to all columns with selective exclusions
normalized_df = df_pivoted.apply(normalize_numeric)
normalized_df

,date,id,meanactivity,meanappCat.builtin,meanappCat.communication,meanappCat.entertainment,meanappCat.finance,meanappCat.game,meanappCat.office,meanappCat.other,...,sumappCat.utilities,sumappCat.weather,sumcall,sumcircumplex.arousal,sumcircumplex.valence,summood,sumscreen,sumsms,Target,Target_date
0,2014-02-17,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.933333,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-18
1,2014-02-18,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-19
2,2014-02-19,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.600000,NaN,NaN,NaN,NaN,-0.875,NaN,2014-02-20
3,2014-02-20,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.933333,NaN,NaN,NaN,NaN,-0.750,NaN,2014-02-21
4,2014-02-21,AS14.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000,NaN,2014-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,2014-05-26,AS14.33,-0.593050,-0.768567,-0.644005,-0.688881,NaN,NaN,0.753773,-0.825962,...,NaN,NaN,-0.400000,-0.157895,-0.285714,0.043478,-0.066081,-0.750,6.2,2014-05-27
1942,2014-05-27,AS14.33,-0.955122,-0.860740,-0.375472,-0.718817,NaN,NaN,NaN,-0.814347,...,-0.938669,NaN,-1.000000,-0.263158,0.000000,0.217391,-0.469077,-0.875,8.2,2014-05-28
1943,2014-05-28,AS14.33,-0.635088,-0.768338,-0.612737,-0.803353,NaN,NaN,-0.714931,-0.696400,...,-0.967024,NaN,-0.400000,0.052632,0.571429,0.652174,-0.019599,-1.000,7.0,2014-05-29
1944,2014-05-29,AS14.33,-0.401755,-0.811438,-0.428989,-0.691209,NaN,NaN,NaN,-0.909371,...,-0.997558,NaN,-0.733333,-0.368421,0.142857,-0.217391,-0.927344,-1.000,6.8,2014-05-30


In [ ]:
# After normalizing drop target and  fill in missing values
normalized_df.dropna(subset='Target', inplace=True)
normalized_df.fillna(0, inplace=True)
normalized_df

,date,id,meanactivity,meanappCat.builtin,meanappCat.communication,meanappCat.entertainment,meanappCat.finance,meanappCat.game,meanappCat.office,meanappCat.other,...,sumappCat.utilities,sumappCat.weather,sumcall,sumcircumplex.arousal,sumcircumplex.valence,summood,sumscreen,sumsms,Target,Target_date
6,2014-02-25,AS14.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,-0.866667,0.000000,0.000000,0.000000,0.000000,0.000,6.250000,2014-02-26
7,2014-02-26,AS14.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,-1.000000,-0.052632,0.142857,-0.043478,0.000000,-0.875,6.333333,2014-02-27
25,2014-03-20,AS14.01,-0.711933,-0.829282,-0.532586,-0.736564,0.000000,0.0,0.000000,-0.957008,...,-0.977687,0.0,-1.000000,0.000000,0.000000,0.000000,-0.704578,0.000,6.200000,2014-03-21
26,2014-03-21,AS14.01,-0.526467,-0.768830,-0.421410,-0.761559,-0.929962,0.0,-0.229090,-0.672040,...,-0.636956,0.0,-0.666667,0.157895,-0.142857,0.217391,0.393158,0.000,6.400000,2014-03-22
27,2014-03-22,AS14.01,-0.163220,-0.856584,-0.457823,-0.790726,-0.918728,0.0,0.000000,-0.859097,...,-0.870360,0.0,-0.866667,0.368421,0.000000,0.260870,-0.202632,-1.000,6.800000,2014-03-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,2014-05-26,AS14.33,-0.593050,-0.768567,-0.644005,-0.688881,0.000000,0.0,0.753773,-0.825962,...,0.000000,0.0,-0.400000,-0.157895,-0.285714,0.043478,-0.066081,-0.750,6.200000,2014-05-27
1942,2014-05-27,AS14.33,-0.955122,-0.860740,-0.375472,-0.718817,0.000000,0.0,0.000000,-0.814347,...,-0.938669,0.0,-1.000000,-0.263158,0.000000,0.217391,-0.469077,-0.875,8.200000,2014-05-28
1943,2014-05-28,AS14.33,-0.635088,-0.768338,-0.612737,-0.803353,0.000000,0.0,-0.714931,-0.696400,...,-0.967024,0.0,-0.400000,0.052632,0.571429,0.652174,-0.019599,-1.000,7.000000,2014-05-29
1944,2014-05-29,AS14.33,-0.401755,-0.811438,-0.428989,-0.691209,0.000000,0.0,0.000000,-0.909371,...,-0.997558,0.0,-0.733333,-0.368421,0.142857,-0.217391,-0.927344,-1.000,6.800000,2014-05-30


In [ ]:
# Calculate the difference
normalized_df['date_difference'] = (normalized_df['Target_date'] - normalized_df['date']).dt.days
 
# Keep rows that predict next day
normalized_df = normalized_df[normalized_df['date_difference'] == 1]

normalized_df.drop(columns='date_difference', inplace=True)

/tmp/ipykernel_24151/2244113363.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df.drop(columns='date_difference', inplace=True)


In [ ]:
### Binning target ###

# Calculate the size of each bin
bins_size = len(df_pivoted['Target']) / 4

# Ensure integer count for indexing (rounding up if not a whole number)
bins_size = int(np.ceil(bins_size))

# Sort DataFrame by 'target'
normalized_df.sort_values(by='Target', inplace=True)

# Create a new column 'bin' that assigns each row to a bin
normalized_df['bin'] = np.floor(np.arange(len(normalized_df)) / bins_size)

# Optionally, you might want to adjust if the last bin has too few items due to rounding
max_bin = normalized_df['bin'].max()
if len(normalized_df[normalized_df['bin'] == max_bin]) < bins_size * 0.5:  # Arbitrary threshold of 50% of bin size
    normalized_df['bin'] = np.floor(np.arange(len(normalized_df)) / (bins_size - 1))

/tmp/ipykernel_24151/1165429632.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df.sort_values(by='Target', inplace=True)
/tmp/ipykernel_24151/1165429632.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df['bin'] = np.floor(np.arange(len(normalized_df)) / bins_size)


In [ ]:
# Calculate min and max for each bin
bin_stats = normalized_df.groupby('bin')['Target'].agg(['min', 'max'])
print(bin_stats)

     min       max
bin               
0.0  3.0  7.000000
1.0  7.0  7.500000
2.0  7.5  9.333333


In [ ]:
# Applying bins using the corrected lambda function
normalized_df['handmatige_bin'] = normalized_df['Target'].apply(
    lambda x: 0 if x < 6 else (1 if x >= 6.4 and x < 7.6 else 2)
)

/tmp/ipykernel_24151/2391876996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df['handmatige_bin'] = normalized_df['Target'].apply(


In [ ]:
normalized_df.columns

,date,id,meanactivity,meanappCat.builtin,meanappCat.communication,meanappCat.entertainment,meanappCat.finance,meanappCat.game,meanappCat.office,meanappCat.other,...,sumcall,sumcircumplex.arousal,sumcircumplex.valence,summood,sumscreen,sumsms,Target,Target_date,bin,handmatige_bin
389,2014-04-19,AS14.07,-0.988959,-0.457522,-0.779373,-0.369591,0.0,0.00000,0.000000,-0.745731,...,0.000000,0.157895,0.142857,0.086957,-0.843047,0.000,3.000000,2014-04-20,0.0,0
394,2014-04-24,AS14.07,-0.441913,-0.720738,-0.648381,0.000000,0.0,0.00000,0.000000,-0.779004,...,-0.866667,-0.263158,0.000000,0.086957,-0.588032,-1.000,3.333333,2014-04-25,0.0,0
661,2014-04-17,AS14.13,-0.853715,-0.802614,-0.502903,-0.666466,0.0,0.00000,-0.551644,-0.400878,...,-0.933333,-0.578947,0.428571,0.260870,0.228184,-1.000,3.500000,2014-04-18,0.0,0
380,2014-04-10,AS14.07,-0.650429,-0.662435,-0.735671,-0.378700,0.0,0.00000,0.000000,-0.927014,...,-0.666667,0.263158,-0.285714,0.217391,-0.796245,-0.625,3.500000,2014-04-11,0.0,0
240,2014-03-23,AS14.05,-0.698879,-0.369469,-0.406583,-0.929967,0.0,0.00000,0.000000,-0.889718,...,-1.000000,-0.684211,-1.000000,-0.173913,-0.605125,0.000,3.750000,2014-03-24,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,2014-04-25,AS14.13,0.556311,-0.257816,-0.740362,0.000000,0.0,0.00000,0.000000,-0.782048,...,0.000000,-0.052632,0.428571,0.217391,-0.403233,0.000,8.800000,2014-04-26,2.0,2
137,2014-04-24,AS14.02,-0.878190,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,...,0.000000,-0.052632,-0.428571,-0.521739,0.000000,0.000,9.000000,2014-04-25,2.0,2
165,2014-03-20,AS14.03,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,...,-0.333333,0.000000,0.000000,0.000000,0.000000,-1.000,9.000000,2014-03-21,2.0,2
1168,2014-04-29,AS14.23,0.319452,-0.544540,-0.768686,-0.851803,0.0,0.00000,0.000000,-0.855359,...,-0.933333,-0.052632,0.571429,0.478261,-0.259494,0.000,9.000000,2014-04-30,2.0,2


In [ ]:
normalized_df.to_csv(f'dataset_{days_in_past}.csv')